## 3.6

In [12]:
# set up directory
dir_counts <- "~/Documents/Dev/SRCourse/original/Applied-Bioinformatics/Unit2-RNAseq/data/htseq_out/day5"
counts_files <- list.files(dir_counts)

In [13]:
# set up sample info
samplesInfo <- as.data.frame(matrix(ncol=2, nrow=length(counts_files)))  
samplesInfo$samplename <- counts_files  
samplesInfo$filename <- counts_files  
samplesInfo$group <- c("mock", "ZIKV", "mock", "ZIKV", "mock", "ZIKV") 
samplesInfo <- samplesInfo[,-c(1:2)]

In [14]:
samplesInfo
class(samplesInfo)

samplename,filename,group
GSM2580321_counts.txt,GSM2580321_counts.txt,mock
GSM2580322_counts.txt,GSM2580322_counts.txt,ZIKV
GSM2580325_counts.txt,GSM2580325_counts.txt,mock
GSM2580326_counts.txt,GSM2580326_counts.txt,ZIKV
GSM2580329_counts.txt,GSM2580329_counts.txt,mock
GSM2580330_counts.txt,GSM2580330_counts.txt,ZIKV


[1] "data.frame"

In [17]:
library("DESeq2")
dds1 <- DESeqDataSetFromHTSeqCount(sampleTable = samplesInfo, 
                                           directory = dir_counts, 
                                           design = ~ group)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”

In [18]:
colData(dds1)

DataFrame with 6 rows and 1 column
                         group
                      <factor>
GSM2580321_counts.txt     mock
GSM2580322_counts.txt     ZIKV
GSM2580325_counts.txt     mock
GSM2580326_counts.txt     ZIKV
GSM2580329_counts.txt     mock
GSM2580330_counts.txt     ZIKV

In [19]:
dds1_deseq <- DESeq(dds1)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [20]:
## First check the sample table we are selecting from
colData(dds1_deseq)

DataFrame with 6 rows and 2 columns
                         group        sizeFactor
                      <factor>         <numeric>
GSM2580321_counts.txt     mock 0.914347097327985
GSM2580322_counts.txt     ZIKV 0.861713044902337
GSM2580325_counts.txt     mock 0.764939157100768
GSM2580326_counts.txt     ZIKV  0.84496827192185
GSM2580329_counts.txt     mock  1.93068645055948
GSM2580330_counts.txt     ZIKV   1.0303145068499

In [22]:
## Specify sample groups that should be compared
res <- results(dds1_deseq, contrast=c("group", "ZIKV", "mock"))
res

log2 fold change (MLE): group ZIKV vs mock 
Wald test p-value: group ZIKV vs mock 
DataFrame with 58735 rows and 6 columns
                         baseMean       log2FoldChange              lfcSE
                        <numeric>            <numeric>          <numeric>
ENSG00000000003  1075.63906962851   -0.292138552730903 0.0965015645303186
ENSG00000000005  6.33853718932102   -0.283335284056721  0.840671238767424
ENSG00000000419   813.68939359019  0.00256337467730136  0.101739836787501
ENSG00000000457  499.648910083186 -0.00215071944894734  0.111152173317996
ENSG00000000460  429.808167012587    -0.10958129780589  0.120593088725289
...                           ...                  ...                ...
ENSG00000285990 0.597527409245946    -1.06736058349928   3.37327270111844
ENSG00000285991   2.3494385123837     1.87259702463587   1.66805197384659
ENSG00000285992                 0                   NA                 NA
ENSG00000285993                 0                   NA         

In [24]:
## sort the gene by adjusted p-value
resSorted <- res[order(res$padj),]
head(resSorted)

log2 fold change (MLE): group ZIKV vs mock 
Wald test p-value: group ZIKV vs mock 
DataFrame with 6 rows and 6 columns
                        baseMean   log2FoldChange             lfcSE
                       <numeric>        <numeric>         <numeric>
ENSG00000213928 512.187721168106 3.27507529008383 0.199747644686085
ENSG00000059378 336.458301796665 3.72099716105068 0.276732956924328
ENSG00000157601 4008.96259771101 5.68040787674336 0.475308620229343
ENSG00000108679 5768.88412825511 1.58820729548322  0.13817689681707
ENSG00000115415  14617.158612877 3.57704283976823 0.333979738711162
ENSG00000187608 3876.78300136491 4.74071973993298 0.450824815194358
                            stat               pvalue                 padj
                       <numeric>            <numeric>            <numeric>
ENSG00000213928 16.3960646205907 2.04034625404368e-60 3.48572754040823e-56
ENSG00000059378 13.4461655829023 3.24255798151487e-41    2.76979302781e-37
ENSG00000157601 11.9509885471938 6.41

In [25]:
# Set up look up API
library("biomaRt")
bm <- useMart(biomart = "ensembl")
bm <- useDataset(dataset = "hsapiens_gene_ensembl", mart = bm)
ens2genesymbol <- getBM(mart = bm, attributes = c('ensembl_gene_id', 'external_gene_name'))

In [36]:
n <- 10
geneList <- list()
for (i in 1 : n) {
        ref <- which(ens2genesymbol$ensembl_gene_id %in% rownames(resSorted)[i])
        name <- ens2genesymbol[ref, "external_gene_name"]
        geneList[[i]] <- ref
        names(geneList)[i] <- name
}

names(geneList)

[1] "IRF9"     "PARP12"   "MX1"      "LGALS3BP" "STAT1"    "ISG15"   
 [7] "OAS3"     "OAS2"     "C5orf56"  "IRF7"